# ETL Project

In [1]:
# Dependencies
import pandas as pd
import numpy as np

import string
import json
import requests
import time

from api_keys import g_api_key, isbn_api_key, goodreads_api_key, username, password
from sqlalchemy import create_engine

## New York Times Bestsellers 2011-2018 CSV
Source: https://data.world/typhon/new-york-times-bestsellers-from-2011-to-2018

### Store CSV into Dataframe

In [2]:
nyt_weeks_df = pd.read_csv("data/nyt_weeks_2011_2018.csv", encoding='cp1252')
nyt_weeks_df

,publisher,dagger,book_review_link,author,primary_isbn10,price,primary_isbn13,sunday_review_link,date,first_chapter_link,contributor,title,age_group,weeks_on_list
0,Riverhead,0,https://www.nytimes.com/2015/01/05/books/the-g...,Paula Hawkins,1594634025,0,9.78E+12,https://www.nytimes.com/2015/02/01/books/revie...,2/19/2017,NaN,by Paula Hawkins,THE GIRL ON THE TRAIN,NaN,102
1,Scribner,0,NaN,Anthony Doerr,1501173219,0,9.78E+12,https://www.nytimes.com/2014/05/11/books/revie...,5/7/2017,NaN,by Anthony Doerr,ALL THE LIGHT WE CANNOT SEE,NaN,81
2,Vintage,0,NaN,E L James,525431888,0,9.78E+12,NaN,3/5/2017,NaN,by E. L. James,FIFTY SHADES DARKER,NaN,66
3,St. Martin's,0,NaN,Kristin Hannah,1466850604,0,9.78E+12,NaN,10/29/2017,NaN,by Kristin Hannah,THE NIGHTINGALE,NaN,63
4,Penguin Group,0,https://www.nytimes.com/2009/02/19/books/19mas...,Kathryn Stockett,1440697663,0,9.78E+12,NaN,4/8/2012,NaN,by Kathryn Stockett,THE HELP,NaN,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2244,Tom Doherty,0,NaN,Robert Jordan and Brandon Sanderson,1429960639,0,9.78E+12,NaN,2/20/2011,NaN,by Robert Jordan and Brandon Sanderson,TOWERS OF MIDNIGHT,NaN,0
2245,Penguin Group,0,NaN,Charles Portis,159020459X,0,9.78E+12,NaN,2/20/2011,NaN,by Charles Portis,TRUE GRIT,NaN,0
2246,Random House,0,NaN,Karen Marie Moning,044033974X,0,9.78E+12,NaN,2/20/2011,NaN,by Karen Marie Moning,SHADOWFEVER,NaN,0
2247,Penguin Group,0,NaN,Nalini Singh,None,0,9.78E+12,NaN,2/13/2011,NaN,by Nalini Singh,ARCHANGELâ€™S CONSORT,NaN,0


### Data Cleaning

In [3]:
# Drop duplicates, if any
nyt_weeks_df = nyt_weeks_df.drop_duplicates()
nyt_weeks_df

,publisher,dagger,book_review_link,author,primary_isbn10,price,primary_isbn13,sunday_review_link,date,first_chapter_link,contributor,title,age_group,weeks_on_list
0,Riverhead,0,https://www.nytimes.com/2015/01/05/books/the-g...,Paula Hawkins,1594634025,0,9.78E+12,https://www.nytimes.com/2015/02/01/books/revie...,2/19/2017,NaN,by Paula Hawkins,THE GIRL ON THE TRAIN,NaN,102
1,Scribner,0,NaN,Anthony Doerr,1501173219,0,9.78E+12,https://www.nytimes.com/2014/05/11/books/revie...,5/7/2017,NaN,by Anthony Doerr,ALL THE LIGHT WE CANNOT SEE,NaN,81
2,Vintage,0,NaN,E L James,525431888,0,9.78E+12,NaN,3/5/2017,NaN,by E. L. James,FIFTY SHADES DARKER,NaN,66
3,St. Martin's,0,NaN,Kristin Hannah,1466850604,0,9.78E+12,NaN,10/29/2017,NaN,by Kristin Hannah,THE NIGHTINGALE,NaN,63
4,Penguin Group,0,https://www.nytimes.com/2009/02/19/books/19mas...,Kathryn Stockett,1440697663,0,9.78E+12,NaN,4/8/2012,NaN,by Kathryn Stockett,THE HELP,NaN,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2244,Tom Doherty,0,NaN,Robert Jordan and Brandon Sanderson,1429960639,0,9.78E+12,NaN,2/20/2011,NaN,by Robert Jordan and Brandon Sanderson,TOWERS OF MIDNIGHT,NaN,0
2245,Penguin Group,0,NaN,Charles Portis,159020459X,0,9.78E+12,NaN,2/20/2011,NaN,by Charles Portis,TRUE GRIT,NaN,0
2246,Random House,0,NaN,Karen Marie Moning,044033974X,0,9.78E+12,NaN,2/20/2011,NaN,by Karen Marie Moning,SHADOWFEVER,NaN,0
2247,Penguin Group,0,NaN,Nalini Singh,None,0,9.78E+12,NaN,2/13/2011,NaN,by Nalini Singh,ARCHANGELâ€™S CONSORT,NaN,0


In [4]:
# Select relevant columns 
nyt_weeks = nyt_weeks_df[["title", "author", "publisher", "date", "weeks_on_list"]]
nyt_weeks

,title,author,publisher,date,weeks_on_list
0,THE GIRL ON THE TRAIN,Paula Hawkins,Riverhead,2/19/2017,102
1,ALL THE LIGHT WE CANNOT SEE,Anthony Doerr,Scribner,5/7/2017,81
2,FIFTY SHADES DARKER,E L James,Vintage,3/5/2017,66
3,THE NIGHTINGALE,Kristin Hannah,St. Martin's,10/29/2017,63
4,THE HELP,Kathryn Stockett,Penguin Group,4/8/2012,58
...,...,...,...,...,...
2244,TOWERS OF MIDNIGHT,Robert Jordan and Brandon Sanderson,Tom Doherty,2/20/2011,0
2245,TRUE GRIT,Charles Portis,Penguin Group,2/20/2011,0
2246,SHADOWFEVER,Karen Marie Moning,Random House,2/20/2011,0
2247,ARCHANGELâ€™S CONSORT,Nalini Singh,Penguin Group,2/13/2011,0


In [5]:
# Replace â€™ with a single apostrophe
nyt_title_new = nyt_weeks.replace("â€™", "'", regex=True)
nyt_title_new

,title,author,publisher,date,weeks_on_list
0,THE GIRL ON THE TRAIN,Paula Hawkins,Riverhead,2/19/2017,102
1,ALL THE LIGHT WE CANNOT SEE,Anthony Doerr,Scribner,5/7/2017,81
2,FIFTY SHADES DARKER,E L James,Vintage,3/5/2017,66
3,THE NIGHTINGALE,Kristin Hannah,St. Martin's,10/29/2017,63
4,THE HELP,Kathryn Stockett,Penguin Group,4/8/2012,58
...,...,...,...,...,...
2244,TOWERS OF MIDNIGHT,Robert Jordan and Brandon Sanderson,Tom Doherty,2/20/2011,0
2245,TRUE GRIT,Charles Portis,Penguin Group,2/20/2011,0
2246,SHADOWFEVER,Karen Marie Moning,Random House,2/20/2011,0
2247,ARCHANGEL'S CONSORT,Nalini Singh,Penguin Group,2/13/2011,0


In [6]:
# Sort descending by weeks on the list 
nyt_title_new.sort_values(by=["weeks_on_list"], ascending=False, inplace=True)
nyt_title_new 

,title,author,publisher,date,weeks_on_list
0,THE GIRL ON THE TRAIN,Paula Hawkins,Riverhead,2/19/2017,102
1,ALL THE LIGHT WE CANNOT SEE,Anthony Doerr,Scribner,5/7/2017,81
2,FIFTY SHADES DARKER,E L James,Vintage,3/5/2017,66
3,THE NIGHTINGALE,Kristin Hannah,St. Martin's,10/29/2017,63
4,THE HELP,Kathryn Stockett,Penguin Group,4/8/2012,58
...,...,...,...,...,...
1681,DEEP STORM,Lincoln Child,Knopf Doubleday Publishing Group,4/12/2015,0
1682,FIFTY SHADES OF GREY,E L James,Vintage,4/5/2015,0
1683,RECKLESS LOVE,Kendall Ryan,Kendall Ryan,4/5/2015,0
1684,ENDANGERED,C J Box,Putnam,4/5/2015,0


In [7]:
# Look at NYT bestsellers on the list for 7 weeks or more (99 books)
nyt_title_new = nyt_title_new.loc[(nyt_title_new["weeks_on_list"] >= 7)]
nyt_title_new

,title,author,publisher,date,weeks_on_list
0,THE GIRL ON THE TRAIN,Paula Hawkins,Riverhead,2/19/2017,102
1,ALL THE LIGHT WE CANNOT SEE,Anthony Doerr,Scribner,5/7/2017,81
2,FIFTY SHADES DARKER,E L James,Vintage,3/5/2017,66
3,THE NIGHTINGALE,Kristin Hannah,St. Martin's,10/29/2017,63
4,THE HELP,Kathryn Stockett,Penguin Group,4/8/2012,58
...,...,...,...,...,...
86,THE DROP,Michael Connelly,"Little, Brown",3/27/2016,7
85,THE WRONG SIDE OF GOODBYE,Michael Connelly,"Little, Brown",1/15/2017,7
84,THE LATE SHOW,Michael Connelly,"Little, Brown",10/15/2017,7
83,THE GIRL WHO TAKES AN EYE FOR AN EYE,David Lagercrantz,Knopf,11/12/2017,7


In [8]:
# Converting title case with capwords to prevent capitalization of a letter after apostrophe
nyt_title_new["title_case"] = nyt_title_new["title"].apply(lambda x: string.capwords(x))
nyt_title_new

C:\Users\ama29\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,title,author,publisher,date,weeks_on_list,title_case
0,THE GIRL ON THE TRAIN,Paula Hawkins,Riverhead,2/19/2017,102,The Girl On The Train
1,ALL THE LIGHT WE CANNOT SEE,Anthony Doerr,Scribner,5/7/2017,81,All The Light We Cannot See
2,FIFTY SHADES DARKER,E L James,Vintage,3/5/2017,66,Fifty Shades Darker
3,THE NIGHTINGALE,Kristin Hannah,St. Martin's,10/29/2017,63,The Nightingale
4,THE HELP,Kathryn Stockett,Penguin Group,4/8/2012,58,The Help
...,...,...,...,...,...,...
86,THE DROP,Michael Connelly,"Little, Brown",3/27/2016,7,The Drop
85,THE WRONG SIDE OF GOODBYE,Michael Connelly,"Little, Brown",1/15/2017,7,The Wrong Side Of Goodbye
84,THE LATE SHOW,Michael Connelly,"Little, Brown",10/15/2017,7,The Late Show
83,THE GIRL WHO TAKES AN EYE FOR AN EYE,David Lagercrantz,Knopf,11/12/2017,7,The Girl Who Takes An Eye For An Eye


In [9]:
# Drop title column
nyt_title_new.drop(["title"], axis = 1, inplace=True)
nyt_title_new

C:\Users\ama29\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,author,publisher,date,weeks_on_list,title_case
0,Paula Hawkins,Riverhead,2/19/2017,102,The Girl On The Train
1,Anthony Doerr,Scribner,5/7/2017,81,All The Light We Cannot See
2,E L James,Vintage,3/5/2017,66,Fifty Shades Darker
3,Kristin Hannah,St. Martin's,10/29/2017,63,The Nightingale
4,Kathryn Stockett,Penguin Group,4/8/2012,58,The Help
...,...,...,...,...,...
86,Michael Connelly,"Little, Brown",3/27/2016,7,The Drop
85,Michael Connelly,"Little, Brown",1/15/2017,7,The Wrong Side Of Goodbye
84,Michael Connelly,"Little, Brown",10/15/2017,7,The Late Show
83,David Lagercrantz,Knopf,11/12/2017,7,The Girl Who Takes An Eye For An Eye


In [10]:
# Rename column title_case to title
nyt_title_new.rename(columns={"title_case": "title"}, inplace=True)
nyt_title_new

C:\Users\ama29\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,author,publisher,date,weeks_on_list,title
0,Paula Hawkins,Riverhead,2/19/2017,102,The Girl On The Train
1,Anthony Doerr,Scribner,5/7/2017,81,All The Light We Cannot See
2,E L James,Vintage,3/5/2017,66,Fifty Shades Darker
3,Kristin Hannah,St. Martin's,10/29/2017,63,The Nightingale
4,Kathryn Stockett,Penguin Group,4/8/2012,58,The Help
...,...,...,...,...,...
86,Michael Connelly,"Little, Brown",3/27/2016,7,The Drop
85,Michael Connelly,"Little, Brown",1/15/2017,7,The Wrong Side Of Goodbye
84,Michael Connelly,"Little, Brown",10/15/2017,7,The Late Show
83,David Lagercrantz,Knopf,11/12/2017,7,The Girl Who Takes An Eye For An Eye


## Google Books API Call 
### Obtaining ISBN13 numbers for each book
Documentation:<br>(1) https://developers.google.com/books/docs/v1/reference/volumes
<br>(2) https://developers.google.com/books/docs/v1/using?authuser=1#auth

In [11]:
# Define get_url function 
def get_url(title, author):
    url = f"https://www.googleapis.com/books/v1/volumes?q={title}+inauthor:{author}&key={g_api_key}"
    return url

In [12]:
# Create two lists, one to hold individual book data and another to hold titles that weren't found
googlebooks_data = []
not_found = []

for n in range(len(nyt_title_new)):
    
    print(f"Processing Book {n + 1}: {nyt_title_new.iloc[n]['title']} by {nyt_title_new.iloc[n]['author']}")

    try: 
        single_book = requests.get(get_url(nyt_title_new.iloc[n]["title"], nyt_title_new.iloc[n]["author"])).json()["items"][0]["volumeInfo"]
         
        # Order of identifiers and available identifiers differ for each book
        # If...elif statement used to obtain ISBN_13 specifically
        for x in range(len(single_book["industryIdentifiers"])):
            if single_book["industryIdentifiers"][x]["type"] == "ISBN_13":
                book_isbn = single_book["industryIdentifiers"][x]["identifier"]
            elif single_book["industryIdentifiers"][x]["type"] == "ISBN_10":
                pass
        
        # Find book data values and save in a dictionary 
        g_book_data = {"title": nyt_title_new.iloc[n]["title"],
                       "author": nyt_title_new.iloc[n]['author'],
                       "isbn": book_isbn
                       }

        # Append copy of dictionary to list so values aren't overwritten
        googlebooks_data.append(g_book_data.copy())
        time.sleep(0.35)
        
    except:
        print("Book not found. Skipping...")
        # Add books not found into separate list
        book_no_isbn = {"title": nyt_title_new.iloc[n]['title'],
                        "author": nyt_title_new.iloc[n]['author']
                        }
        not_found.append(book_no_isbn)
    
print("-------------------------------")  
print("Data Retrieval Complete")
print("-------------------------------")

Processing Book 1: The Girl On The Train by Paula Hawkins
Processing Book 2: All The Light We Cannot See by Anthony Doerr
Processing Book 3: Fifty Shades Darker by E L James
Processing Book 4: The Nightingale by Kristin Hannah
Processing Book 5: The Help by Kathryn Stockett
Processing Book 6: A Man Called Ove by Fredrik Backman
Processing Book 7: Milk And Honey by Rupi Kaur
Processing Book 8: A Game Of Thrones by George R R Martin
Processing Book 9: Big Little Lies by Liane Moriarty
Processing Book 10: Before We Were Yours by Lisa Wingate
Processing Book 11: The Woman In Cabin 10 by Ruth Ware
Processing Book 12: The Girl With The Dragon Tattoo by Stieg Larsson
Processing Book 13: Little Fires Everywhere by Celeste Ng
Processing Book 14: The Handmaid's Tale by Margaret Atwood
Processing Book 15: The Whistler by John Grisham
Processing Book 16: The Longest Ride by Nicholas Sparks
Processing Book 17: Ready Player One by Ernest Cline
Processing Book 18: Camino Island by John Grisham
Proces

In [14]:
# Chttp://localhost:8888/notebooks/etl_project/etl_project.ipynb#reate dataframe
googlebooks_df = pd.DataFrame(googlebooks_data)
# Check counts for dataframe (96 books found)
googlebooks_df.count()

title     96
author    96
isbn      96
dtype: int64

In [15]:
# Check to make sure no duplicate ISBNs
googlebooks_df["isbn"].nunique()

96

In [16]:
# Create dataframe for books that weren't found
isbndb_other = pd.DataFrame(not_found)
# Check counts for dataframe
isbndb_other

,title,author
0,The Liar,Nora Roberts
1,Kill Alex Cross,James Patterson
2,The Best Of Me,Nicholas Sparks


In [17]:
# Create new column "isbn"
isbndb_other["isbn"] = pd.Series()
isbndb_other

C:\Users\ama29\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


,title,author,isbn
0,The Liar,Nora Roberts,NaN
1,Kill Alex Cross,James Patterson,NaN
2,The Best Of Me,Nicholas Sparks,NaN


## ISBNdb API call
### Finding ISBN13 numbers for books not found with Google API

In [18]:
# Define get_url function 
def get_url(title, author):
    url = f"https://api2.isbndb.com/books/{title}{author}"
    return url

h = {"Authorization": isbn_api_key}

In [19]:
# Go through each book and look up its ISBN13
for n in range(len(isbndb_other)):
    
    print(f"Searching for {isbndb_other.iloc[n]['title']} by {isbndb_other.iloc[n]['author']}")

    try: 
        single_book = requests.get(get_url(isbndb_other.iloc[n]["title"], isbndb_other.iloc[n]["author"]), headers=h).json()
        single_book_isbn = single_book["books"][0]["isbn13"]
        isbndb_other.iloc[n, isbndb_other.columns.get_loc("isbn")] = single_book_isbn
        time.sleep(0.3)

    except:
        print("Book not found. Skipping...")
             
print("-------------------------------")  
print("Data Retrieval Complete")
print("-------------------------------")

Searching for The Liar by Nora Roberts
Searching for Kill Alex Cross by James Patterson
Searching for The Best Of Me by Nicholas Sparks
-------------------------------
Data Retrieval Complete
-------------------------------


In [20]:
# View updated table
isbndb_other

,title,author,isbn
0,The Liar,Nora Roberts,9781607060673
1,Kill Alex Cross,James Patterson,9781611132168
2,The Best Of Me,Nicholas Sparks,9780751542981


In [21]:
# Append isbndb_other dataframe to googlebooks_df
googlebooks_df = googlebooks_df.append(isbndb_other)
googlebooks_df

,title,author,isbn
0,The Girl On The Train,Paula Hawkins,9780698185395
1,All The Light We Cannot See,Anthony Doerr,9781476746586
2,Fifty Shades Darker,E L James,9780525431886
3,The Nightingale,Kristin Hannah,9781628995015
4,The Help,Kathryn Stockett,9781440697661
...,...,...,...
94,The Girl Who Takes An Eye For An Eye,David Lagercrantz,9780451494337
95,Revival,Stephen King,9781476770383
0,The Liar,Nora Roberts,9781607060673
1,Kill Alex Cross,James Patterson,9781611132168


In [22]:
# Reset index
googlebooks_df.reset_index(drop=True, inplace=True)
googlebooks_df

,title,author,isbn
0,The Girl On The Train,Paula Hawkins,9780698185395
1,All The Light We Cannot See,Anthony Doerr,9781476746586
2,Fifty Shades Darker,E L James,9780525431886
3,The Nightingale,Kristin Hannah,9781628995015
4,The Help,Kathryn Stockett,9781440697661
...,...,...,...
94,The Girl Who Takes An Eye For An Eye,David Lagercrantz,9780451494337
95,Revival,Stephen King,9781476770383
96,The Liar,Nora Roberts,9781607060673
97,Kill Alex Cross,James Patterson,9781611132168


#### Please keep in mind that the ISBNs obtained via API are not foolproof. In this case, "The Liar" by Nora Roberts and "The Target" by David Baldacci were both found to have incorrect ISBNs, which were manually corrected.

In [23]:
# Manually change ISBN of "The Liar" by Nora Roberts
googlebooks_df["isbn"].loc[googlebooks_df["title"] == "The Liar"] = 9780399170867
# Manually change ISBN of "The Target" by David Baldacci
googlebooks_df["isbn"].loc[googlebooks_df["title"] == "The Target"] = 9781455521203

In [24]:
# Confirm modification of one
googlebooks_df["isbn"].loc[googlebooks_df["title"] == "The Liar"]

96    9780399170867
Name: isbn, dtype: object

In [25]:
# Confirm no null values in isbn column
googlebooks_df["isbn"].isna().sum()

0

In [26]:
# Join googlebooks_df on "title" and "author" with nyt_title_new for isbn column
nyt_merge_df = pd.merge(googlebooks_df, nyt_title_new, on=["title", "author"])
nyt_merge_df

,title,author,isbn,publisher,date,weeks_on_list
0,The Girl On The Train,Paula Hawkins,9780698185395,Riverhead,2/19/2017,102
1,All The Light We Cannot See,Anthony Doerr,9781476746586,Scribner,5/7/2017,81
2,Fifty Shades Darker,E L James,9780525431886,Vintage,3/5/2017,66
3,The Nightingale,Kristin Hannah,9781628995015,St. Martin's,10/29/2017,63
4,The Help,Kathryn Stockett,9781440697661,Penguin Group,4/8/2012,58
...,...,...,...,...,...,...
94,The Girl Who Takes An Eye For An Eye,David Lagercrantz,9780451494337,Knopf,11/12/2017,7
95,Revival,Stephen King,9781476770383,Scribner,1/11/2015,7
96,The Liar,Nora Roberts,9780399170867,Putnam,4/10/2016,10
97,Kill Alex Cross,James Patterson,9781611132168,"Little, Brown & Company",2/5/2012,10


In [27]:
# Sort descending by weeks on the list
nyt_merge_df.sort_values(by=["weeks_on_list"], ascending=False, inplace=True)
nyt_merge_df.reset_index(drop=True, inplace=True)
nyt_merge_df 

,title,author,isbn,publisher,date,weeks_on_list
0,The Girl On The Train,Paula Hawkins,9780698185395,Riverhead,2/19/2017,102
1,All The Light We Cannot See,Anthony Doerr,9781476746586,Scribner,5/7/2017,81
2,Fifty Shades Darker,E L James,9780525431886,Vintage,3/5/2017,66
3,The Nightingale,Kristin Hannah,9781628995015,St. Martin's,10/29/2017,63
4,The Help,Kathryn Stockett,9781440697661,Penguin Group,4/8/2012,58
...,...,...,...,...,...,...
94,Hopeless,Colleen Hoover,9781481251884,Colleen Hoover,2/24/2013,7
95,Mr. Mercedes,Stephen King,9781476754451,Scribner,8/3/2014,7
96,King And Maxwell,David Baldacci,9781455521227,Grand Central,1/19/2014,7
97,A Week In Winter,Maeve Binchy,9780385350082,Knopf Doubleday Publishing,4/14/2013,7


## Goodreads API 
#### Call with ISBN13 to retrieve further data (rating, review numbers, etc.)
Documentation: https://www.goodreads.com/api/index#book.show_by_isbn

In [28]:
# Define get_url_gr function 
def get_url_gr(isbn):
    url = f"https://www.goodreads.com/book/review_counts.json?isbns={isbn}&key={goodreads_api_key}"
    return url

In [29]:
# Go through each book and find its average rating, total ratings count, and total text reviews count
goodreads_data = []

for n in range(len(nyt_merge_df)):
    
    print(f"Processing Book {n + 1}: {nyt_merge_df.iloc[n]['title']} by {nyt_merge_df.iloc[n]['author']}")
    
    try: 
        gr_book_json = requests.get(get_url_gr(nyt_merge_df.iloc[n]["isbn"])).json()["books"][0]
        # Find book data values and save in a dictionary 
        gr_book_data = {"isbn": nyt_merge_df.iloc[n]["isbn"],
                        "avg_rating": gr_book_json["average_rating"],
                        "ratings_count": gr_book_json["work_ratings_count"],
                        "reviews_count": gr_book_json["work_text_reviews_count"]
                       }
        # Append copy of dictionary to list so values aren't overwritten
        goodreads_data.append(gr_book_data.copy())
        # Goodreads API guidelines: no more than one call per second
        time.sleep(1.5)

    except:
        print("Book not found. Skipping...")

print("-------------------------------")  
print("Data Retrieval Complete")
print("-------------------------------")

Processing Book 1: The Girl On The Train by Paula Hawkins
Processing Book 2: All The Light We Cannot See by Anthony Doerr
Processing Book 3: Fifty Shades Darker by E L James
Processing Book 4: The Nightingale by Kristin Hannah
Processing Book 5: The Help by Kathryn Stockett
Processing Book 6: A Man Called Ove by Fredrik Backman
Processing Book 7: Milk And Honey by Rupi Kaur
Processing Book 8: A Game Of Thrones by George R R Martin
Processing Book 9: Big Little Lies by Liane Moriarty
Processing Book 10: Before We Were Yours by Lisa Wingate
Processing Book 11: The Woman In Cabin 10 by Ruth Ware
Processing Book 12: Little Fires Everywhere by Celeste Ng
Processing Book 13: The Girl With The Dragon Tattoo by Stieg Larsson
Processing Book 14: The Handmaid's Tale by Margaret Atwood
Processing Book 15: The Whistler by John Grisham
Processing Book 16: The Longest Ride by Nicholas Sparks
Processing Book 17: Ready Player One by Ernest Cline
Processing Book 18: Camino Island by John Grisham
Proces

In [30]:
# Create dataframe
goodreads_df = pd.DataFrame(goodreads_data)
# Check counts to ensure all books in dataframe
goodreads_df.count()

isbn             99
avg_rating       99
ratings_count    99
reviews_count    99
dtype: int64

In [31]:
# View final table
goodreads_df

,isbn,avg_rating,ratings_count,reviews_count
0,9780698185395,3.92,2078549,109889
1,9781476746586,4.33,1003524,74999
2,9780525431886,3.84,736764,29076
3,9781628995015,4.58,678152,63280
4,9781440697661,4.47,2138643,84467
...,...,...,...,...
94,9781481251884,4.29,264516,21462
95,9781476754451,3.97,235432,16602
96,9781455521227,4.10,28509,1898
97,9780385350082,3.83,42860,5408


In [32]:
# Export goodreads_df as .csv file
goodreads_df.to_csv("output_data/goodreads.csv", encoding='utf-8', index = False)

## Postgresql

### Connect to local database (books_db)

In [33]:
rds_conn_string = f"{username}:{password}@localhost:5432/books_db"
engine = create_engine(f'postgresql://{rds_conn_string}')

### Check for tables

In [34]:
engine.table_names()

['nyt_weeks', 'goodreads_ratings']

### Use Pandas to load dataframes into database

In [35]:
nyt_merge_df.to_sql(name='nyt_weeks', con=engine, if_exists='append', index=False)

In [36]:
goodreads_df.to_sql(name='goodreads_ratings', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the New York Times Bestsellers table 

In [37]:
pd.read_sql_query('select * from nyt_weeks', con=engine).head()

,title,author,isbn,publisher,date,weeks_on_list
0,The Girl On The Train,Paula Hawkins,9780698185395,Riverhead,2017-02-19,102
1,All The Light We Cannot See,Anthony Doerr,9781476746586,Scribner,2017-05-07,81
2,Fifty Shades Darker,E L James,9780525431886,Vintage,2017-03-05,66
3,The Nightingale,Kristin Hannah,9781628995015,St. Martin's,2017-10-29,63
4,The Help,Kathryn Stockett,9781440697661,Penguin Group,2012-04-08,58


### Confirm data has been added by querying the Goodreads table 

In [38]:
pd.read_sql_query('select * from goodreads_ratings', con=engine).head()

,isbn,avg_rating,ratings_count,reviews_count
0,9780698185395,3.92,2078549,109889
1,9781476746586,4.33,1003524,74999
2,9780525431886,3.84,736764,29076
3,9781628995015,4.58,678152,63280
4,9781440697661,4.47,2138643,84467
